In [105]:
import pandas as pd
import numpy as np

In [61]:
df_HospDis = pd.read_excel(r"C:\Users\chand\OneDrive\Documents\GitHub\Team18_Matlib_marvels_PythonHackathon_Apr24\Cardiac_Outcomes.xlsx",
                                         sheet_name = 'Hospitalization_Discharge')


In [62]:
df_HospDis.head()

,inpatient_number,DestinationDischarge,admission_ward,admission_way,discharge_department,visit_times,respiratory_support,oxygen_inhalation,dischargeDay,Admission_date,...,death_within_28_days,re_admission_within_28_days,death_within_3_months,re_admission_within_3_months,death_within_6_months,re_admission_within_6_months,time_of_death__days_from_admission,readmission_time_days_from_admission,return_to_emergency_department_within_6_months,time_to_emergency_department_within_6_months
0,857781,Home,Cardiology,NonEmergency,Cardiology,1,NaN,OxygenTherapy,11,2017-01-24,...,0,0,0,0,0,0,NaN,NaN,0.0,NaN
1,743087,Home,Cardiology,NonEmergency,Cardiology,1,NaN,OxygenTherapy,8,2017-05-05,...,0,0,0,0,0,0,NaN,NaN,0.0,NaN
2,866418,Home,Cardiology,NonEmergency,Cardiology,2,NaN,OxygenTherapy,5,2016-11-18,...,0,0,0,0,0,0,NaN,NaN,0.0,NaN
3,775928,Home,Cardiology,Emergency,Cardiology,1,NaN,OxygenTherapy,11,2017-10-02,...,0,1,0,1,0,1,NaN,19.0,1.0,19.0
4,810128,Home,Cardiology,NonEmergency,Cardiology,1,NaN,OxygenTherapy,5,2019-11-17,...,0,0,0,0,0,0,NaN,NaN,0.0,NaN


In [7]:
df_CardiacComp = pd.read_excel(r"C:\Users\chand\OneDrive\Documents\GitHub\Team18_Matlib_marvels_PythonHackathon_Apr24\Cardiac_Outcomes.xlsx",
                                         sheet_name = 'CardiacComplications')
df_CardiacComp.head()

,inpatient_number,NYHA_cardiac_function_classification,Killip_grade,myocardial_infarction,congestive_heart_failure,peripheral_vascular_disease,type_of_heart_failure,LVEF,left_ventricular_end_diastolic_diameter_LV,mitral_valve_EMS,mitral_valve_AMS,EA,tricuspid_valve_return_velocity,tricuspid_valve_return_pressure
0,857781,3,3,0,0,0,Both,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,743087,3,1,0,0,0,Both,NaN,40.0,1.16,1.52,NaN,3.34,47.0
2,866418,2,2,0,0,0,Both,NaN,46.0,0.84,0.12,7.00,2.80,32.0
3,775928,3,2,0,0,0,Both,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,810128,3,2,0,0,0,Both,NaN,64.0,1.20,0.90,1.33,NaN,NaN


In [8]:
df_respo = pd.read_excel(r"C:\Users\chand\OneDrive\Documents\GitHub\Team18_Matlib_marvels_PythonHackathon_Apr24\Cardiac_Outcomes.xlsx",
                                         sheet_name = 'Responsivenes')
df_respo.head()

,inpatient_number,eye_opening,verbal_response,movement,consciousness,GCS
0,857781,4,5,6,Clear,15
1,743087,4,5,6,Clear,15
2,866418,4,5,6,Clear,15
3,775928,4,5,6,Clear,15
4,810128,4,5,6,Clear,15


In [9]:
df_PatientHistory = pd.read_excel(r"C:\Users\chand\OneDrive\Documents\GitHub\Team18_Matlib_marvels_PythonHackathon_Apr24\Cardiac_Outcomes.xlsx",
                                         sheet_name = 'PatientHistory')
df_PatientHistory.head()

,inpatient_number,cerebrovascular_disease,dementia,Chronic_obstructive_pulmonary_disease,connective_tissue_disease,peptic_ulcer_disease,diabetes,moderate_to_severe_chronic_kidney_disease,hemiplegia,leukemia,malignant_lymphoma,solid_tumor,liver_disease,AIDS,CCI_score,type_II_respiratory_failure,acute_renal_failure
0,857781,0,0,1,0,0.0,1,0.0,0,0,0,0,0.0,0,2.0,NonTypeII,0
1,743087,0,0,0,0,0.0,0,0.0,0,0,0,0,0.0,0,0.0,NonTypeII,0
2,866418,0,0,0,0,0.0,0,0.0,0,0,0,0,0.0,0,0.0,NonTypeII,0
3,775928,0,0,1,0,0.0,0,1.0,0,0,0,0,0.0,0,2.0,NonTypeII,0
4,810128,0,0,0,0,0.0,0,0.0,0,0,0,0,0.0,0,0.0,NonTypeII,0


In [11]:
df_labs = pd.read_excel(r"C:\Users\chand\OneDrive\Documents\GitHub\Team18_Matlib_marvels_PythonHackathon_Apr24\Cardiac_Outcomes.xlsx",
                                         sheet_name = 'Labs')
df_labs.head()

,inpatient_number,body_temperature,pulse,respiration,systolic_blood_pressure,diastolic_blood_pressure,map,fio2,creatinine_enzymatic_method,urea,...,measured_residual_base,measured_bicarbonate,carboxyhemoglobin,body_temperature_blood_gas,oxygen_saturation,partial_oxygen_pressure,oxyhemoglobin,anion_gap,free_calcium,total_hemoglobin
0,857781,36.7,87,19,102,64,76.666667,33,108.3,12.55,...,-2.1,21.2,0.4,37.0,97.0,93.0,95.9,17.8,1.14,125.0
1,743087,36.8,95,18,150,70,96.666667,33,62.0,4.29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,866418,36.5,98,18,102,67,78.666667,33,185.1,15.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,775928,36.0,73,19,110,74,86.000000,33,104.8,8.16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,810128,35.0,88,19,134,62,86.000000,33,83.9,6.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_demography = pd.read_excel(r"C:\Users\chand\OneDrive\Documents\GitHub\Team18_Matlib_marvels_PythonHackathon_Apr24\Cardiac_Outcomes.xlsx",
                                         sheet_name = 'Demography')
df_demography.head()

,inpatient_number,gender,weight,height,BMI,occupation,ageCat
0,857781,Male,50.0,1.64,18.590125,UrbanResident,69-79
1,743087,Female,51.0,1.63,19.195303,UrbanResident,69-79
2,866418,Male,70.0,1.70,24.221453,farmer,59-69
3,775928,Male,65.0,1.70,22.491349,UrbanResident,69-79
4,810128,Female,76.0,1.55,31.633715,UrbanResident,69-79


In [15]:
df_PatientPresc = pd.read_excel(r"C:\Users\chand\OneDrive\Documents\GitHub\Team18_Matlib_marvels_PythonHackathon_Apr24\Cardiac_Outcomes.xlsx",
                                         sheet_name = 'Patient_Precriptions')

df_PatientPresc.head()

,inpatient_number,Drug_name
0,857781,sulfotanshinone sodium injection
1,857781,Furosemide tablet
2,857781,Enoxaparin Sodium injection
3,857781,Meglumine Adenosine Cyclophosphate for injection
4,857781,Furosemide injection


## 33:Retrieve the row with minimum white_blood_cell count using sorting technique.


In [32]:
sort_wbc=df_labs.sort_values(by=['white_blood_cell'])
sort_wbc[['inpatient_number','white_blood_cell']].iloc[0]



inpatient_number    746068.00
white_blood_cell         1.66
Name: 865, dtype: float64

In [33]:
sort_wbc.iloc[0]


inpatient_number           746068.00
body_temperature               37.10
pulse                          71.00
respiration                    20.00
systolic_blood_pressure       130.00
                             ...    
partial_oxygen_pressure        95.00
oxyhemoglobin                  96.30
anion_gap                      16.80
free_calcium                    1.11
total_hemoglobin               94.00
Name: 865, Length: 107, dtype: float64

## 34: get the last letter of discharge_department of patients using RegEX

In [64]:
discharge_dpt=df_HospDis['discharge_department']
last_letter=discharge_dpt.str.extract('(\w\Z)')
df_HospDis['last_letter']=last_letter
#df_HD
df_HospDis[['discharge_department','last_letter']]

,discharge_department,last_letter
0,Cardiology,y
1,Cardiology,y
2,Cardiology,y
3,Cardiology,y
4,Cardiology,y
...,...,...
2003,GeneralWard,d
2004,GeneralWard,d
2005,Cardiology,y
2006,Cardiology,y


## 35:find the way to reverse string for given input (eg: input : This hackathon is about cardiac outcomes: outcomes cardiac about is hackathon This)

In [60]:
str='This hackathon is about cardiac outcomes'
#split the string into words
revstr=str.split()
print(revstr)
#reverse the words
revstr.reverse()
print(revstr)
#join the words with a space in between
' '.join(revstr)

['This', 'hackathon', 'is', 'about', 'cardiac', 'outcomes']
['outcomes', 'cardiac', 'about', 'is', 'hackathon', 'This']


'outcomes cardiac about is hackathon This'

## 37: Using dischargeDay values print even if it even number and odd if it odd number

In [79]:
#disday= df_HospDis['dischargeDay']
#df_HospDis[['dischargeDay', 'disday']]



if (df_HospDis['dischargeDay'])/2 == 0:
    #df_HospDis['disdayevenodd']=disday
    print("dischargeDay is even")
else:
    print("dischargeDay is odd")
    



ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [120]:
#df_HospDis['dischargeDay']= disday
#if df_HospDis['dischargeDay']!= 0:
#if df_HospDis['dischargeDay'] % 2 == 0:
#        print('even')
df_HospDis['dischargeDay'] % 2


0       1
1       0
2       1
3       1
4       1
       ..
2003    1
2004    0
2005    0
2006    1
2007    0
Name: dischargeDay, Length: 2008, dtype: int64

In [121]:
for day in df_HospDis['dischargeDay']:
    if day != 0 and day % 2 == 0:
        print(f"{day} is even")
    elif day == 0:
        print(f"{day} is neither even nor odd")
    else:
        print(f"{day} is odd")

11 is odd
8 is even
5 is odd
11 is odd
5 is odd
6 is even
10 is even
27 is odd
20 is even
4 is even
9 is odd
9 is odd
11 is odd
22 is even
8 is even
3 is odd
16 is even
14 is even
7 is odd
10 is even
5 is odd
11 is odd
7 is odd
9 is odd
9 is odd
8 is even
8 is even
5 is odd
11 is odd
7 is odd
7 is odd
10 is even
15 is odd
4 is even
9 is odd
6 is even
7 is odd
7 is odd
7 is odd
4 is even
3 is odd
5 is odd
8 is even
4 is even
9 is odd
9 is odd
11 is odd
19 is odd
6 is even
7 is odd
15 is odd
4 is even
18 is even
3 is odd
11 is odd
14 is even
12 is even
11 is odd
9 is odd
8 is even
7 is odd
2 is even
9 is odd
30 is even
8 is even
6 is even
3 is odd
5 is odd
21 is odd
8 is even
7 is odd
8 is even
5 is odd
5 is odd
5 is odd
7 is odd
10 is even
6 is even
10 is even
7 is odd
8 is even
11 is odd
9 is odd
15 is odd
7 is odd
5 is odd
7 is odd
6 is even
5 is odd
26 is even
11 is odd
3 is odd
7 is odd
7 is odd
2 is even
14 is even
12 is even
6 is even
10 is even
4 is even
9 is odd
9 is odd
8 is ev

## 38:find the length of Tuple for any given input (Hint: Input a string from user )

In [104]:
# taking integer values as input and using split method
x=input("Enter multiples of 5: ").split()

# converting List into Tuple
tuple(x)

print(x)

print("Length of the tuple is: ", len(x))

Enter multiples of 5: 10 15 25
['10', '15', '25']
Length of the tuple is:  3


In [102]:
# taking string values as input
str=input("Enter few names of fruits: ")

# using split method to break each entry
a=str.split()

b= tuple(a)
print (b)
print ("Length of the tuple is:", len(b))


Enter few names of fruits: aaa sss sss ss
('aaa', 'sss', 'sss', 'ss')
Length of the tuple is: 4


## 40: Select 25 different random values between 200 to 800.

In [106]:
#Generating a list of random integers using numpy.random.randint function
print(list(np.random.randint(low=200,high=801,size=25)))

[675, 224, 796, 570, 368, 250, 635, 319, 735, 789, 394, 280, 455, 217, 288, 555, 270, 620, 712, 326, 409, 464, 321, 449, 356]


## 48: Using loops, create a multiplication table for the number 7

In [112]:
# Using for loop and range function
for x in range(1,11):
    result=x * 7
    print(x," * 7 = ", result)
    
 

1  * 7 =  7
2  * 7 =  14
3  * 7 =  21
4  * 7 =  28
5  * 7 =  35
6  * 7 =  42
7  * 7 =  49
8  * 7 =  56
9  * 7 =  63
10  * 7 =  70


## 47:Generate a 3x3 Array using Random values between 1to 9 and Fix the values and position in Next Run.

In [115]:
from numpy import random

x = random.choice([1,2,3,4,5,6,7,8, 9], size=(3, 3))

print(x)

[[3 7 5]
 [2 6 2]
 [2 4 1]]


In [116]:
# Seed function is used to get the same set of random numbers 
np.random.seed(0)
print(x)


[[3 7 5]
 [2 6 2]
 [2 4 1]]


In [117]:
from sklearn.preprocessing import LabelEncoder
